<a href="https://colab.research.google.com/github/ErickMM98/network_clustering_parallel/blob/main/foo_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!find / -iname 'libdevice'
!find / -iname 'libnvvm.so'

/usr/local/lib/python3.7/dist-packages/jaxlib/cuda/nvvm/libdevice
/usr/local/cuda-10.0/nvvm/libdevice
/usr/local/cuda-11.0/nvvm/libdevice
/usr/local/cuda-10.1/nvvm/libdevice
/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so
/usr/local/cuda-11.0/nvvm/lib64/libnvvm.so
/usr/local/cuda-10.1/nvvm/lib64/libnvvm.so


In [ ]:
import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/local/cuda-10.1/nvvm/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/local/cuda-10.1/nvvm/lib64/libnvvm.so"

In [ ]:
from numba import cuda
import numba
print(cuda.gpus)

<Managed Device 0>


In [ ]:
import random
# biblioteca de Numba
from numba import jit

#¡¡¡descomentar para optimizar, te vas a sorprender!!!
@jit(nopython=True)
def get_clustering(D):
  #t_diag = np.diag(D**3)
  #degree_vector = np.matrix([D[i][0].sum() for i in range(n) ])
  #degree_vector_minus_one = degree_vector - 1
  #divisor = np.array(degree_vector) * np.array(degree_vector_minus_one)
  print(np.diag(np.linalg.matrix_power(D, 3)) )#/ divisor)

# t_diag = np.diag(D**3)
# degree_vector = np.matrix([D[i][0].sum() for i in range(n) ])
# degree_vector_minus_one = degree_vector - 1
# divisor = np.array(degree_vector) * np.array(degree_vector_minus_one)
# print(t_diag / divisor)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import time 
import matplotlib.pyplot as plt
#G=nx.karate_club_graph()

G = nx.Graph()

G.add_edges_from([(1,2),
                  (2,3),
                  (3,4),
                  (4,5),
                  (5,2),
                  (4,2)])
n_g = 15
p = 0.5
G = nx.generators.random_graphs.binomial_graph(n_g, p)
n = G.number_of_nodes()

D = nx.adjacency_matrix(G)
D = D.toarray()
D = np.matrix(D) 
D = np.array(D)
print(D**3)
t_i = time.time()
get_clustering(D)
t_f = time.time()
t_m = t_f - t_i

print(t_m)
#print(degree_vector_minus_one)
#print(G.adj.items() )


#nodes_nbrs = ((n, G[n]) for n in G.nbunch_iter(nodes))
nodes_nbrs = G.adj.items()

t_i = time.time()
#list_foo = []
#print(nx.clustering(G))
"""
for v, v_nbrs in nodes_nbrs:
    #print(v,v_nbrs)
    vs = set(v_nbrs) - {v}
    #print(vs)
    #print(set(G[v]))
    gen_degree = Counter(len(vs & (set(G[w]) - {w})) for w in vs)
    #print(gen_degree)
    ntriangles = sum(k * val for k, val in gen_degree.items())
    list_foo.append(ntriangles)
    #print(ntriangles)
    #yield (v, len(vs), ntriangles, gen_degree)
"""    
t_f = time.time()
t_m = t_f - t_i
#print(list_foo)
print(t_m)

# Pruebas

In [14]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import time 
import matplotlib.pyplot as plt


n_g = 800
p = 0.5
G = nx.generators.random_graphs.binomial_graph(n_g, p)
n = G.number_of_nodes()

D = nx.adjacency_matrix(G)
D = D.toarray()
D = np.matrix(D) 
D = np.array(D)
#print(D**3
t_i = time.time()
nx.clustering(G)
#print(np.linalg.matrix_power(D,3))
t_f = time.time()
t_m = t_f - t_i

print(t_m)

15.120669603347778


#Pruebas con C 

In [ ]:
# variable de tipo String que en si es el programa
codigo = """
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

/*
const int N = 3;

void foo_function(int arr[N][N]){
    int i, j;
    for (i = 0; i < N; i++)
      for (j = 0; j < N; j++)
        arr[i][j] = 0;
    	printf("%d \t", arr[i][j]);
}
*/

int main(int argc, char const *argv[]){
	int n = 3000;
	int D[n][n];
	float p = 0.8;
	int i,j;
	int **A;
	//foo_function(matrix_a);
	
	//Aquí generamos la matriz
	for (i = 0; i < n; i++){
		for(j= 0 ; j < n; j++){
			D[i][j] = 0;
		}
	}

	for (i = 0; i < n; i++){
		for(j= i +1 ; j < n; j++){
			double r = ((double) rand() / (RAND_MAX));
			printf(" %f \n ", r);
			if ( r < p){
					D[i][j] = 1;
					D[j][i] = 1;
			}
		}
	}

	//Hasta aquí tenemos la matriz de adyacencia


	//Esto es para imprimir la matriz
	/*
	for (i = 0; i < n; i++){
		for(j= 0; j < n; j++){
			printf(%d \t", D[i][j]);
		}
		printf("------------ \n");
	}
	*/


	//#Primer algoritmo secuencial.
	int number_triagles[n];
	int number_degree[n];
	float clustering[n];

	//Sacamos el vector de grados
	for (int node = 0; node < n; ++node){
		int degree = 0;
		for (int k = 0; k < n; ++k){
			degree += D[node][k];
		}
		number_degree[node] = degree;
		//printf("Node %d ------> %d \n",node, degree );
	}

	//Sacamos los triángulos
	for (int node = 0; node < n; ++node){
		//printf("%ls\n", D[node]);
		number_triagles[node] = 0;
		for (int v = 0; v < n; ++v){
			int v_comun_v = 0;
			if ( node != v){
				//printf("%d , %d \n", node, v );
				//Conteo de triángulos
				if (D[node][v] == 1){
					for (int k = 0; k < n; ++k){
						v_comun_v += D[node][k] * D[v][k];
					}
				}
			
			}
			//Este es el doble de números al cuadrado
			number_triagles[node] += v_comun_v;
		}
		//typeof(D[node][node]);
		//printf("%d  --------- %d \n", node+1, number_triagles[node]);
	}

	//Sacamos clustering
	for (int node = 0; node < n; ++node){
		float t = (float) number_triagles[node];
		float d = (float) number_degree[node];
		//printf("%f \n", t / (d * (d-1)) );
	}
	//Acaterminaclustering
	
}
"""

In [ ]:
# se crea el archivo con permisos para escribir mediante python
archivo_texto = open("hola.c", "w")
# se escribe el programa en el archivo 
archivo_texto.write(codigo)
# se cierra el buffer de escritura
archivo_texto.close()

# Secuencial

In [49]:
!gcc -o sequencial main_sequencial.c
!./sequencial

n , time 
5 , 0.000004 
25 , 0.000129 
45 , 0.000588 
65 , 0.001803 
85 , 0.004119 
105 , 0.007838 
125 , 0.013512 
145 , 0.021025 
165 , 0.030514 
185 , 0.049236 
205 , 0.068397 
225 , 0.094327 
245 , 0.108564 
265 , 0.139467 
285 , 0.168929 
305 , 0.220251 
325 , 0.273975 
345 , 0.317544 
365 , 0.384724 
385 , 0.430228 
405 , 0.475293 
425 , 0.611068 
445 , 0.776042 
465 , 0.874660 
485 , 0.995597 
505 , 1.244076 
525 , 1.331676 
545 , 1.749010 
565 , 1.823776 
585 , 2.101964 
605 , 3.526283 
625 , 2.670388 
645 , 5.459533 
665 , 4.577064 
685 , 8.193859 
705 , 5.753129 
725 , 3.999679 
745 , 5.338454 
765 , 6.196447 
785 , 7.929352 
805 , 6.336577 
825 , 5.606652 


In [ ]:
!cat foo.c

In [8]:
!./sequencial

# Paralelo

In [50]:
!gcc -fopenmp main_matrix_reduc.c -o paralelo 
!./paralelo

n , time 
5 , 0.000204 
25 , 0.000034 
45 , 0.000237 
65 , 0.000555 
85 , 0.001144 
105 , 0.002165 
125 , 0.003580 
145 , 0.010062 
165 , 0.010148 
185 , 0.013376 
205 , 0.018192 
225 , 0.038533 
245 , 0.032408 
265 , 0.039940 
285 , 0.042025 
305 , 0.047724 
325 , 0.058087 
345 , 0.069724 
365 , 0.078618 
385 , 0.093936 
405 , 0.105096 
425 , 0.115954 
445 , 0.137800 
465 , 0.155134 
485 , 0.184177 
505 , 0.221253 
525 , 0.241837 
545 , 0.265034 
565 , 0.300564 
585 , 0.296677 
605 , 0.327319 
625 , 0.362426 
645 , 0.394845 
665 , 0.437432 
685 , 0.466999 
705 , 0.515914 
725 , 0.554233 
745 , 0.606060 
765 , 0.651224 
785 , 0.722073 
805 , 0.759316 
825 , 0.814273 
845 , 0.935313 
865 , 0.950354 
885 , 1.013006 
905 , 1.159940 
925 , 1.151070 
945 , 1.224905 
965 , 1.299853 
985 , 1.400441 
1005 , 1.469819 
